In [1]:
from datasets import load_dataset
dataset = load_dataset("tianharjuno/twitter-parse", cache_dir="cache/")

README.md: 0.00B [00:00, ?B/s]

data/source_stage_1-00000-of-00001.parqu(…):   0%|          | 0.00/26.2M [00:00<?, ?B/s]

data/source_stage_2-00000-of-00001.parqu(…):   0%|          | 0.00/32.3M [00:00<?, ?B/s]

data/cleaned-00000-of-00001.parquet:   0%|          | 0.00/25.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/755k [00:00<?, ?B/s]

...

Generating source_stage_1 split:   0%|          | 0/201583 [00:00<?, ? examples/s]

Generating source_stage_2 split:   0%|          | 0/247820 [00:00<?, ? examples/s]

Generating cleaned split:   0%|          | 0/195952 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/19999 [00:00<?, ? examples/s]

In [2]:
from datasets import ClassLabel
class_labels = ClassLabel(2, ["Irrelevant", "Relevant"])
train_ds = dataset["train"].select_columns(["content", "relevant"])
train_ds = train_ds.rename_column("relevant", "label")
train_ds = train_ds.cast_column("label", class_labels)

train_ds

Casting the dataset:   0%|          | 0/19999 [00:00<?, ? examples/s]

Dataset({
    features: ['content', 'label'],
    num_rows: 19999
})

In [4]:
import re
# We are NOT importing replace_word_elongation anymore
from indoNLP.preprocessing import emoji_to_words

def clean_tweet_for_nusabert(row):
    text = row['content']

    # 1. Lowercase
    text = text.lower()

    # 2. Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)

    # 3. Remove mentions and RT
    # This regex is safe and does not affect 'ruu tni'
    text = re.sub(r'rt @\S+|@\S+', '', text)

    # 4. Remove hashtags (keep the word)
    text = re.sub(r'#(\S+)', r'\1', text)

    # 5. Convert emojis to words (Preserves sentiment)
    text = emoji_to_words(text)

    # 6. Normalize word elongation (CUSTOM, SAFER REGEX)
    # This replaces 3 or more repeated chars (e.g., 'bangeeet' -> 'banget')
    # It will NOT affect 'uu' or 'ruu', fixing your bug.
    text = re.sub(r'(\w)\1{2,}', r'\1', text)

    # 7. Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    row["content"] = text
    return row

In [5]:
sentence_train_ds = train_ds.map(clean_tweet_for_nusabert)

Map:   0%|          | 0/19999 [00:00<?, ? examples/s]

In [21]:
sentence_train_ds["content"]

Column(['hasil dr uu tni yg baru..', 'sudah saatnya kita dukung ruu tni agar prajurit bisa cepat bertindak dan negara aman', 'mari kita lihat bersama, kuat mana? ambisi atau anjloknya rupiah ininegrikriminal indonesiagelap darkindonesia darkpik2', 'gini : revisi uu tni, berdasarkan wacana dan draf yang sedang dibahas hingga saat ini, tidak secara eksplisit bertujuan menjadikan tni sebagai lembaga berpolitik. tni tetap diatur sebagai alat negara di bidang pertahanan, sesuai dengan pasal 30 uud 1945 dan uu no. 34 tahun 2004', '"menyerang! menyerang! al-qassam. setiap petugas dan setiap pengecut.” situasi demonstrasi besar-besaran hari ini di ibu kota yordania, amman untuk mendukung gaza dan mujahidin.'])

In [6]:
from sentence_transformers import SentenceTransformer
sentence_transformer = SentenceTransformer("LazarusNLP/all-nusabert-large-v4",
                                           cache_folder="cache/",
                                           device="cuda",
                                           )
embeddings = sentence_transformer.encode(sentence_train_ds["content"], show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True, device="cuda", batch_size=512)



modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/176 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

In [8]:
from lib import DreamCluster
clusterer = DreamCluster("stability", sample_size=10000, with_sampling=True)
clusterer.fit(embeddings)

ModuleNotFoundError: No module named 'lib'